In [1]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-7qyu0iub
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-7qyu0iub
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.8 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=e5ffe6b01bccf4cdaae66746eeb630b7d05c3e6e4d239ec55323c235410ccb6c
  Stored in directory: /tmp/pip-ephem-wheel-cache-dhbq86x8/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [2]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 30.4 MB/s eta 0:00:00


In [9]:
import torch
import clip
import numpy as np
from PIL import Image
from pymongo import MongoClient
import requests
from io import BytesIO
from pymongo.server_api import ServerApi

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [5]:
from google.colab import userdata

In [10]:
username = userdata.get('MONGO_USERNAME')
password = userdata.get('MONGO_PASSWORD')
cluster = userdata.get('MONGO_CLUSTER')
database = userdata.get('MONGO_DATABAS')

In [11]:
db_url = f"mongodb+srv://{username}:{password}@{cluster}/?retryWrites=true&w=majority"
client = MongoClient(db_url, server_api=ServerApi('1'))
try:
    client.admin.command('ping')
    print("Connected")
except Exception as e:
    print(e)

Connected


In [12]:
db=client[database]
products_collection=db['products']

In [13]:
products=list(products_collection.find({'image_url':{'$exists':True}}))

In [14]:
from tqdm import tqdm

In [16]:
for i,p in tqdm(enumerate(products),total=len(products)):
  try:
    response=requests.get(p['image_url'],timeout=10)
    img=Image.open(BytesIO(response.content)).convert('RGB')

    img_pp=preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
      clp=model.encode_image(img_pp)
      clp=clp.cpu().numpy().flatten()

    norm=np.linalg.norm(clp)
    if norm>0:
      clp=clp/norm

    products_collection.update_one({'_id':p['_id']},
                                   {'$set':{'clip_embedding':clp.tolist()}})

  except Exception as e:
    print(e)


100%|██████████| 225/225 [02:22<00:00,  1.58it/s]
